In [1]:
#Testing funtionalities of Findsym

In [2]:
import mechanize as mech
from casase import casread
import numpy as np
from numpy import linalg as la

In [3]:
br = mech.Browser()
br.set_handle_robots(False)

In [4]:
file = "Ca3Ti2O7_A21am.cell"
Atoms = casread(file)

In [5]:
#Get attributes
nAtoms = Atoms.get_global_number_of_atoms()
elems = ' '.join(Atoms.get_chemical_symbols())
cell = Atoms.get_cell()
latvecs = [' '.join([str(c) for c in cell[i, :]])+'\r\n' for i in range(3)]
posns = Atoms.get_scaled_positions()
positions = [' '.join([str(p) for p in posns[i, :]])+'\r\n' for i in range(nAtoms)]

spins = Atoms.get_initial_magnetic_moments()
spins = np.random.rand(nAtoms)

In [6]:
br.open('http://stokes.byu.edu/iso/findsym.php')

<response_seek_wrapper at 0x119bf5190 whose wrapped object = <closeable_response at 0x11a41a8d0 whose fp = <_io.BufferedReader>>>

In [7]:
br.form = list(br.forms())[1]

In [8]:
br['title'] = file

In [9]:
lattol = str(1e-3)
postol = str(1e-3)
magtol = str(1e-2)

In [10]:
br['acclat'] = lattol
br['accpos'] = postol
br['accmag'] = magtol

In [11]:
#Raw positions and spins
n = [0, 1, 0] #Magnetic moment direction
proj_spins = np.array([[spins[i]*np.dot(np.array(n), cell[j]) / \
    np.dot(cell[j], cell[j]) for j in range(3)] for i in range(nAtoms)])

In [12]:
spatiomag_positions = np.zeros((nAtoms, 6))
spatiomag_positions[:, :3] = posns
spatiomag_positions[:,3:] = proj_spins

In [13]:
spatiomag_positions = np.zeros((nAtoms, 6))
spatiomag_positions[:, :3] = posns
spatiomag_positions[:,3:] = proj_spins
positions = [' '.join([str(p) for p in spatiomag_positions[i, :]])+'\r\n' for i in range(nAtoms)]

In [14]:
origin=2
axeso='abc'
axesm='ab(c)'
index=None
format=None
br.find_control("magnetic").items[0].selected=True


In [15]:
br['title'] = file.replace(".cell", "")
br['axeso'] = [axeso]
br['axesm'] = [axesm]
br['vectors'] = ''.join(latvecs)
br['atoms'] = str(nAtoms)
br['types'] = elems
br['origin'] = [str(origin)]

In [16]:
br['positions'] = ''.join(positions)

In [17]:
response = br.submit()

In [18]:
output = response.read().decode('utf-8')
print(output)

<html>
<head>
<title>FINDSYM</title>
</head>
<body BGCOLOR="#FFFFFF">
<pre>
FINDSYM, Version 7.0, October 2019
Written by Harold T. Stokes, Branton J. Campbell, and Dorian M. Hatch
Brigham Young University
 
Ca3Ti2O7_A21am
Tolerance of lattice:    0.001000
Tolerance of atomic position:    0.001000
Tolerance of magnetic moment:    0.010000
Lattice vectors in cartesian coordinates:
   5.44770   0.00000   0.00000
   0.00000   5.38764   0.00000
   0.00000   0.00000  19.29005
Lattice parameters, a,b,c,alpha,beta,gamma:
   5.44770   5.38764  19.29005  90.00000  90.00000  90.00000
Centering: P
Number of atoms in unit cell:
  48
Type of each atom:
28*O 12*Ca 8*Ti 
         Position                     Magnetic moment        
   1 O   0.04976  0.67560  0.00000    0.000  0.111  0.000  
   2 O   0.54976  0.32440  0.00000    0.000  0.144  0.000  
   3 O   0.04976  0.17560  0.50000    0.000  0.131  0.000  
   4 O   0.54976  0.82440  0.50000    0.000  0.101  0.000  
   5 O   0.01073  0.31715  0.6978

In [19]:
cifstart = '# CIF file created by FINDSYM'
cifend = '# end of cif'
cifFile = output[output.index(cifstart):output.index(cifend) + len(cifend)]

In [20]:
cifFile
cifLines = cifFile.splitlines()
for iLine, line in enumerate(cifLines):
        if '_space_group_magn.number_BNS' in line:
            SG_number = line.split(" ")[1]
        elif '_space_group_magn.name_BNS' in line:
            SG_name = line.split('"')[1]
print('Space group: ', SG_number, SG_name)

Space group:  "1.1" P1


In [23]:
testname = "a.b.c.in"
cifname = testname.replace(testname.split(".")[-1], "cif")

In [24]:
with open(cifname, "w+") as CIF:
    CIF.write(cifFile)


In [28]:
with open("a.b.c.cif") as CIFINPUT:
    data = CIFINPUT.read()
    
print(data)

# CIF file created by FINDSYM, version 7.0

data_findsym-output
_audit_creation_method FINDSYM
 
_cell_length_a    5.3876381000
_cell_length_b    5.4477001000
_cell_length_c    19.2900528000
_cell_angle_alpha 90.0000000000
_cell_angle_beta  90.0000000000
_cell_angle_gamma 90.0000000000
_cell_volume      566.1676140177
 
_space_group_magn.number_BNS "1.1"
_space_group_magn.name_BNS "P1"
_space_group_magn.number_OG "1.1.1"
_space_group_magn.name_OG "P1"
_space_group_magn.point_group_name "1"
 
loop_
_space_group_magn_transforms.id
_space_group_magn_transforms.Pp_abc
_space_group_magn_transforms.source
1 a,b,c;0,0,0 "BNS"
2 a,b,c;0,0,0 "OG"
 
loop_
_space_group_symop_magn_operation.id
_space_group_symop_magn_operation.xyz
1 x,y,z,+1 
 
loop_
_space_group_symop_magn_centering.id
_space_group_symop_magn_centering.xyz
1 x,y,z,+1 
 
loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_symmetry_multiplicity
_atom_site_Wyckoff_label
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
_